# シンプルテスト

In [1]:
import msal
import requests

TENANT_ID = "58970528-d083-476f-9082-2d21bd11a800"
CLIENT_ID = "99a8342c-e187-44bd-a6d6-3c9ad5ffd0f0"

# Scopes must match the delegated permissions you added in Entra ID
SCOPES = [
    "Calendars.Read",
    "Chat.Read",
    "OnlineMeetings.Read",
    "OnlineMeetingTranscript.Read.All",
    "Team.ReadBasic.All",
    "Channel.ReadBasic.All",
    "ChannelMessage.Read.All",
    "User.Read",
]

AUTHORITY = f"https://login.microsoftonline.com/{TENANT_ID}"

app = msal.PublicClientApplication(
    client_id=CLIENT_ID,
    authority=AUTHORITY,)

flow = app.initiate_device_flow(scopes=SCOPES)
if "user_code" not in flow:
    raise RuntimeError(f"Failed to create device flow: {flow}")

print(flow["message"])  # User instructions

result = app.acquire_token_by_device_flow(flow)

if "access_token" not in result:
    raise RuntimeError(f"Failed to acquire token: {result}")

access_token = result["access_token"]

headers = {"Authorization": f"Bearer {access_token}"}

# Example 1: List chats the user is in
r = requests.get("https://graph.microsoft.com/v1.0/me/chats", headers=headers)
print("me/chats:", r.status_code)
print(r.text)

# Example 2: Joined teams
r = requests.get("https://graph.microsoft.com/v1.0/me/joinedTeams", headers=headers)
print("me/joinedTeams:", r.status_code)
print(r.text)


To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code AM4WYUJNQ to authenticate.
me/chats: 200
{"@odata.context":"https://graph.microsoft.com/v1.0/$metadata#chats","@odata.count":8,"value":[{"id":"19:83b23399b2f94e90a771c8c5fe262fa5@thread.v2","topic":null,"createdDateTime":"2026-01-29T06:10:00Z","lastUpdatedDateTime":"2026-01-29T06:10:00.513Z","chatType":"group","webUrl":"https://teams.microsoft.com/l/chat/19%3A83b23399b2f94e90a771c8c5fe262fa5%40thread.v2/0?tenantId=58970528-d083-476f-9082-2d21bd11a800","tenantId":"58970528-d083-476f-9082-2d21bd11a800","isHiddenForAllMembers":false,"onlineMeetingInfo":null,"viewpoint":{"isHidden":false,"lastMessageReadDateTime":"2026-01-29T06:44:06.78Z"}},{"id":"19:a0576424-04db-4808-88d9-4cfaa09b02c6_a52ac978-0df2-4e13-91e9-a1571eecde73@unq.gbl.spaces","topic":null,"createdDateTime":"2026-01-29T06:08:57Z","lastUpdatedDateTime":"2026-01-29T06:08:57.138Z","chatType":"oneOnOne","webUrl":"https://teams.micros

# Teams チャネル/グループチャット/会議データの取得

このセクションでは、すでに取得済みの `access_token` / `headers` を使って、次の情報を取得するコードを追加します。



- 特定チーム配下のチャネル一覧とチャネル ID 取得

- グループチャット & 会議チャットの ID 取得

- 特定チャネルの全メッセージ（＋必要であればスレッド返信）

- 特定グループチャットの全メッセージ

- ユーザーが主催/参加しているオンライン会議のメタデータ & 参加者

- 会議トランスクリプト一覧 & コンテンツ取得（権限がある場合）

- 会議に紐づくチャット（chatType = `meeting`）のメッセージ取得



※ 実際に API を呼び出すには、Microsoft Graph 側で必要な権限が付与されている必要があります。権限不足の場合は 403 などのエラーになります。

In [2]:
import json

from typing import List, Dict, Any



# 共通: @odata.nextLink をたどって全件取得するヘルパー

def get_with_paging(url: str, headers: Dict[str, str], params: Dict[str, Any] | None = None) -> List[Dict[str, Any]]:

    items: List[Dict[str, Any]] = []

    while url:

        print(f"GET {url}")

        resp = requests.get(url, headers=headers, params=params)

        print("  ->", resp.status_code)
        print(resp.text)

        resp.raise_for_status()

        data = resp.json()

        items.extend(data.get("value", []))

        # 次ページがある場合は @odata.nextLink に URL が入る

        url = data.get("@odata.nextLink")

        # 2 ページ目以降は params は不要

        params = None

    return items



# 1. チーム一覧 & 各チーム配下のチャネル一覧を取得して ID を確認

teams_url = "https://graph.microsoft.com/v1.0/me/joinedTeams"

teams = get_with_paging(teams_url, headers)



print("==== Joined Teams ====")

for t in teams:

    print(f"Team: {t.get('displayName')} | id={t.get('id')}")



print("\n==== Channels per Team ====")

for t in teams:

    team_id = t.get("id")

    channels_url = f"https://graph.microsoft.com/v1.0/teams/{team_id}/channels"

    channels = get_with_paging(channels_url, headers)

    print(f"\nTeam: {t.get('displayName')} ({team_id})")

    for ch in channels:

        print(f"  Channel: {ch.get('displayName')} | id={ch.get('id')}")



# 2. グループチャット & 会議チャットの ID 一覧

#   chatType: 'oneOnOne' | 'group' | 'meeting'

group_chats_url = "https://graph.microsoft.com/v1.0/me/chats?$filter=chatType eq 'group'"

group_chats = get_with_paging(group_chats_url, headers)



print("\n==== Group Chats (chatType = group) ====")

for chat in group_chats:

    print(f"Group Chat: {chat.get('topic')} | id={chat.get('id')} | lastMessagePreview={chat.get('lastMessagePreview', {}).get('body', {}).get('content')}")



meeting_chats_url = "https://graph.microsoft.com/v1.0/me/chats?$filter=chatType eq 'meeting'"

meeting_chats = get_with_paging(meeting_chats_url, headers)



print("\n==== Meeting Chats (chatType = meeting) ====")

for chat in meeting_chats:

    # onlineMeetingInfo や topic からどの会議かを識別できます

    print("Meeting Chat:")

    print("  id=", chat.get("id"))

    print("  topic=", chat.get("topic"))

    print("  onlineMeetingInfo=", json.dumps(chat.get("onlineMeetingInfo"), ensure_ascii=False))

    print()

GET https://graph.microsoft.com/v1.0/me/joinedTeams
  -> 200
{"@odata.context":"https://graph.microsoft.com/v1.0/$metadata#teams","@odata.count":1,"value":[{"id":"a7c4b767-03be-4974-9230-4d62ec88ccb2","createdDateTime":null,"displayName":"contoso_policy","description":"demo: contoso用の社内ポリシー共有チーム","internalId":null,"classification":null,"specialization":null,"visibility":null,"webUrl":null,"isArchived":false,"tenantId":"58970528-d083-476f-9082-2d21bd11a800","isMembershipLimitedToOwners":null,"memberSettings":null,"guestSettings":null,"messagingSettings":null,"funSettings":null,"discoverySettings":null,"tagSettings":null,"summary":null}]}
==== Joined Teams ====
Team: contoso_policy | id=a7c4b767-03be-4974-9230-4d62ec88ccb2

==== Channels per Team ====
GET https://graph.microsoft.com/v1.0/teams/a7c4b767-03be-4974-9230-4d62ec88ccb2/channels
  -> 200
{"@odata.context":"https://graph.microsoft.com/v1.0/$metadata#teams('a7c4b767-03be-4974-9230-4d62ec88ccb2')/channels","@odata.count":2,"value"

## 特定チャネルの全メッセージ取得

上のセルで確認した `team_id` と `channel_id` を使って、特定チャネルの全メッセージ（＋必要に応じて返信）を取得します。



- メッセージ本体: `/teams/{team-id}/channels/{channel-id}/messages`

- 各メッセージの返信: `/teams/{team-id}/channels/{channel-id}/messages/{message-id}/replies`



取得したメッセージは Python のリストとして返すので、後続で pandas に流し込んだり、ファイル保存したりできます。

In [3]:
from datetime import datetime



def get_channel_messages(team_id: str, channel_id: str) -> List[Dict[str, Any]]:

    """特定チャネルの全メッセージを取得"""

    url = f"https://graph.microsoft.com/v1.0/teams/{team_id}/channels/{channel_id}/messages"

    return get_with_paging(url, headers)



def get_channel_message_replies(team_id: str, channel_id: str, message_id: str) -> List[Dict[str, Any]]:

    """特定メッセージのスレッド返信を全件取得"""

    url = f"https://graph.microsoft.com/v1.0/teams/{team_id}/channels/{channel_id}/messages/{message_id}/replies"

    return get_with_paging(url, headers)



# 使い方例: team_id / channel_id は上のセルの出力からコピペ

# team_id = "<チームID>"

# channel_id = "<チャネルID>"

# messages = get_channel_messages(team_id, channel_id)

# print(len(messages), "messages")

# 先頭 3 件をタイトル＋送信者＋送信日時だけ表示

# for m in messages[:3]:

#     from_user = (m.get("from", {}) or {}).get("user", {}) or {}

#     user_name = from_user.get("displayName")

#     created = m.get("createdDateTime")

#     print(created, "|", user_name, "|", (m.get("body", {}) or {}).get("content"))

In [4]:
# 使い方例: team_id / channel_id は上のセルの出力からコピペ

team_id = "a7c4b767-03be-4974-9230-4d62ec88ccb2"

channel_id = "19:8zjMe4C0P-zr7EqzETew2ebgu1c6w6bNX-HVQ8Mmovc1@thread.tacv2"

messages = get_channel_messages(team_id, channel_id)

print(len(messages), "messages")

# 先頭 3 件をタイトル＋送信者＋送信日時だけ表示

for m in messages:

    from_user = (m.get("from", {}) or {}).get("user", {}) or {}

    user_name = from_user.get("displayName")

    created = m.get("createdDateTime")

    print(created, "|", user_name, "|", (m.get("body", {}) or {}).get("content"))

GET https://graph.microsoft.com/v1.0/teams/a7c4b767-03be-4974-9230-4d62ec88ccb2/channels/19:8zjMe4C0P-zr7EqzETew2ebgu1c6w6bNX-HVQ8Mmovc1@thread.tacv2/messages
  -> 200
{"@odata.context":"https://graph.microsoft.com/v1.0/$metadata#teams('a7c4b767-03be-4974-9230-4d62ec88ccb2')/channels('19%3A8zjMe4C0P-zr7EqzETew2ebgu1c6w6bNX-HVQ8Mmovc1%40thread.tacv2')/messages","@odata.count":16,"value":[{"id":"1767581607302","replyToId":null,"etag":"1769660632981","messageType":"message","createdDateTime":"2026-01-05T02:53:27.302Z","lastModifiedDateTime":"2026-01-29T04:23:52.981Z","lastEditedDateTime":null,"deletedDateTime":null,"subject":null,"summary":null,"chatId":null,"importance":"normal","locale":"en-us","webUrl":"https://teams.microsoft.com/l/message/19%3A8zjMe4C0P-zr7EqzETew2ebgu1c6w6bNX-HVQ8Mmovc1%40thread.tacv2/1767581607302?groupId=a7c4b767-03be-4974-9230-4d62ec88ccb2&tenantId=58970528-d083-476f-9082-2d21bd11a800&createdTime=1767581607302&parentMessageId=1767581607302","policyViolation":null

## 特定グループチャットの全メッセージ取得

グループチャット ID（上の `Group Chats` の一覧で表示される `id`）を指定して、そのチャットの全メッセージを取得します。



- メッセージ本体: `/chats/{chat-id}/messages`

- 1 対 1 チャットでもグループチャットでも同じエンドポイントで取得できます。



ここでは主に `chatType = group` のチャットを想定しています。

In [5]:
def get_chat_messages(chat_id: str) -> List[Dict[str, Any]]:

    """特定チャット (1:1 / グループ / 会議チャット) の全メッセージを取得"""

    url = f"https://graph.microsoft.com/v1.0/chats/{chat_id}/messages"

    return get_with_paging(url, headers)



# 使い方例: group_chat_id は Group Chats の出力からコピペ

# group_chat_id = "<グループチャットID>"

# chat_messages = get_chat_messages(group_chat_id)

# print(len(chat_messages), "messages in group chat")

# for m in chat_messages[:3]:

#     from_user = (m.get("from", {}) or {}).get("user", {}) or {}

#     user_name = from_user.get("displayName")

#     created = m.get("createdDateTime")

#     print(created, "|", user_name, "|", (m.get("body", {}) or {}).get("content"))

In [6]:
# 使い方例: group_chat_id は Group Chats の出力からコピペ

group_chat_id = "19:83b23399b2f94e90a771c8c5fe262fa5@thread.v2"

chat_messages = get_chat_messages(group_chat_id)

print(len(chat_messages), "messages in group chat")

for m in chat_messages:

    from_user = (m.get("from", {}) or {}).get("user", {}) or {}

    user_name = from_user.get("displayName")

    created = m.get("createdDateTime")

    print(created, "|", user_name, "|", (m.get("body", {}) or {}).get("content"))

GET https://graph.microsoft.com/v1.0/chats/19:83b23399b2f94e90a771c8c5fe262fa5@thread.v2/messages
  -> 200
{"@odata.context":"https://graph.microsoft.com/v1.0/$metadata#chats('19%3A83b23399b2f94e90a771c8c5fe262fa5%40thread.v2')/messages","@odata.count":6,"value":[{"id":"1769669046780","replyToId":null,"etag":"1769669046780","messageType":"message","createdDateTime":"2026-01-29T06:44:06.78Z","lastModifiedDateTime":"2026-01-29T06:44:06.78Z","lastEditedDateTime":null,"deletedDateTime":null,"subject":null,"summary":null,"chatId":"19:83b23399b2f94e90a771c8c5fe262fa5@thread.v2","importance":"normal","locale":"en-us","webUrl":null,"channelIdentity":null,"policyViolation":null,"eventDetail":null,"from":{"application":null,"device":null,"user":{"@odata.type":"#microsoft.graph.teamworkUserIdentity","id":"a0576424-04db-4808-88d9-4cfaa09b02c6","displayName":"Hiroaki Ishida (IC)","userIdentityType":"aadUser","tenantId":"58970528-d083-476f-9082-2d21bd11a800"}},"body":{"contentType":"html","content":

## オンライン会議のメタデータ・参加者・トランスクリプト・会議チャット

オンライン会議まわりの情報取得は次のように行います。



1. `/me/onlineMeetings` でユーザーが主催/参加するオンライン会議一覧を取得

    - `id`, `subject`, `startDateTime`, `endDateTime`, `joinWebUrl` などが含まれます

    - `participants` の中に参加者情報があります

2. 特定会議に対して `/me/onlineMeetings/{meeting-id}/transcripts` でトランスクリプト一覧

    - さらに `/me/onlineMeetings/{meeting-id}/transcripts/{transcript-id}/content` で実際のテキスト (通常は VTT) を取得

    - これらの API はテナント側でトランスクリプト機能が有効 & 必要な Graph 権限が付与されている必要があります

3. 会議中チャットは、`chatType = 'meeting'` のチャットとして `/me/chats` から取得可能

    - 上の「Meeting Chats」一覧の `id` を `get_chat_messages` に渡せば、その会議チャットの全メッセージを取得できます



> 備考: トランスクリプト関連 API は組織全体/app 権限や RSC (resource-specific consent) 権限を前提としていることが多いため、

> 単純なユーザー委任トークンだけだと 403 になる場合があります。その場合はテナント管理者に Graph 権限設定を相談してください。

In [13]:
from urllib.parse import quote

from datetime import datetime



def get_online_meeting_by_join_url(join_url: str) -> Dict[str, Any] | None:

    """joinWebUrl をキーに単一の onlineMeeting を取得



    /me/onlineMeetings は『一覧』ではなくフィルタ取得が基本で、

    例: GET /me/onlineMeetings?$filter=JoinWebUrl eq '{joinWebUrl}'



    join_url は Meeting チャットの onlineMeetingInfo や、

    カレンダー イベントの onlineMeeting プロパティから取得できます。

    """



    url = "https://graph.microsoft.com/v1.0/me/onlineMeetings"

    # requests に任せて $filter をクエリとして渡す

    flt = f"JoinWebUrl eq '{join_url}'"

    params = {"$filter": flt}



    print("GET", url, "? $filter=", flt)

    resp = requests.get(url, headers=headers, params=params)

    print("  ->", resp.status_code)

    print(resp.text)

    resp.raise_for_status()



    data = resp.json()

    meetings = data.get("value", []) or []

    if not meetings:

        print("No onlineMeeting found for this joinWebUrl")

        return None

    if len(meetings) > 1:

        print("Warning: multiple onlineMeetings returned. Using the first one.")

    return meetings[0]



def show_meeting_participants(meeting: Dict[str, Any]) -> None:

    """onlineMeeting オブジェクトから参加者情報をざっくり表示"""

    participants = meeting.get("participants", {}) or {}

    organizer = participants.get("organizer", {}) or {}

    attendees = participants.get("attendees", []) or []



    print("Organizer:")

    print("  ", (organizer.get("upn") or organizer.get("identity", {}).get("user", {}).get("displayName")))



    print("Attendees:")

    for a in attendees:

        identity = a.get("upn") or (a.get("identity", {}).get("user", {}) or {}).get("displayName")

        role = a.get("role")

        print("  ", identity, "| role=", role)



def list_meeting_transcripts(meeting_id: str) -> List[Dict[str, Any]]:

    """特定オンライン会議のトランスクリプト一覧を取得"""

    url = f"https://graph.microsoft.com/v1.0/me/onlineMeetings/{meeting_id}/transcripts"

    resp = requests.get(url, headers=headers)

    print("GET", url, "->", resp.status_code)

    print(resp.text)

    resp.raise_for_status()

    data = resp.json()

    transcripts = data.get("value", [])

    for t in transcripts:

        print("Transcript id=", t.get("id"), "| created=", t.get("createdDateTime"))

    return transcripts



def download_meeting_transcript_content_vtt(meeting_id: str, transcript_id: str, out_path: str) -> None:
    """
    onlineMeetings の transcript を VTT でダウンロードして保存
    """
    url = f"https://graph.microsoft.com/v1.0/me/onlineMeetings/{meeting_id}/transcripts/{transcript_id}/content"
    params = {"$format": "text/vtt"}

    hdrs = dict(headers)
    hdrs["Accept"] = "text/vtt"

    resp = requests.get(url, headers=hdrs, params=params)
    print("GET", resp.url, "->", resp.status_code)
    print(resp.text[:500])  # 失敗時の原因確認用（成功時はVTTの先頭が出る）

    resp.raise_for_status()

    # content はバイナリで返ることがあるので wb が安全
    with open(out_path, "wb") as f:
        f.write(resp.content)

    print("Saved transcript content to", out_path)



def download_meeting_transcript_content(meeting_id: str, transcript_id: str, out_path: str) -> None:

    """単一トランスクリプトの content を取得してファイル保存 (VTT など)

    NOTE: 一部テナント/エンドポイントでは /me パスで /content にアクセスすると 400 になるため、

          実運用では callTranscripts API (download_meeting_transcript_content_via_call_transcript) の利用を推奨します。"""

    url = f"https://graph.microsoft.com/v1.0/me/onlineMeetings/{meeting_id}/transcripts/{transcript_id}/content"

    resp = requests.get(url, headers=headers)

    print("GET", url, "->", resp.status_code)

    print(resp.text[:500])

    resp.raise_for_status()

    with open(out_path, "w", encoding="utf-8") as f:

        f.write(resp.text)

    print("Saved transcript content to", out_path)



def download_meeting_transcript_content_by_url(transcript_content_url: str, out_path: str) -> None:

    """transcriptContentUrl を使ってトランスクリプトの content を取得してファイル保存

    ※ このエンドポイントも環境によっては 400 になる場合があります。"""

    resp = requests.get(transcript_content_url, headers=headers)

    print("GET", transcript_content_url, "->", resp.status_code)

    print(resp.text[:500])

    resp.raise_for_status()

    with open(out_path, "w", encoding="utf-8") as f:

        f.write(resp.text)

    print("Saved transcript content to", out_path)



# 使い方例:

# 1. Meeting チャット一覧セルの出力から、対象会議の onlineMeetingInfo.joinWebUrl をコピペして join_url に入れる

#    （またはカレンダーイベントの onlineMeeting.joinUrl でも可）

# join_url = "<オンライン会議の joinWebUrl>"

# meeting = get_online_meeting_by_join_url(join_url)

# if meeting:

#     show_meeting_participants(meeting)

#     m_id = meeting.get("id")

#     transcripts = list_meeting_transcripts(m_id)

#     if transcripts:

#         t0_id = transcripts[0].get("id")

#         download_meeting_transcript_content_vtt(m_id, t0_id, "meeting_transcript_0.vtt")



# 会議中チャットのメッセージは、上の meeting_chats の出力から対象チャットの id を選び、

#   meeting_chat_id = "<Meeting Chat ID>"

#   meeting_chat_messages = get_chat_messages(meeting_chat_id)

# のように呼び出します。

In [15]:
# 使い方例:

# 1. meeting_chats 一覧セルを実行して、対象会議の onlineMeetingInfo を確認

# 2. その中の joinWebUrl を下の join_url にペースト



join_url = "https://teams.microsoft.com/l/meetup-join/19%3ameeting_YzkyZGE0NmItN2Y4Zi00NmJkLTgxYzktZDlmZGY3OTM2MGRm%40thread.v2/0?context=%7b%22Tid%22%3a%2258970528-d083-476f-9082-2d21bd11a800%22%2c%22Oid%22%3a%22a52ac978-0df2-4e13-91e9-a1571eecde73%22%7d"

meeting = get_online_meeting_by_join_url(join_url)

if meeting:

    show_meeting_participants(meeting)

    m_id = meeting.get("id")

    transcripts = list_meeting_transcripts(m_id)

    if transcripts:

        # transcripts[0]["id"] は callTranscript の ID として使える

        t0_id = transcripts[0].get("id")

        if t0_id:

            download_meeting_transcript_content_vtt(m_id, t0_id, "meeting_transcript_0.vtt")

        else:

            print("transcript id がレスポンスに含まれていません")



# 会議中チャットのメッセージは、上の meeting_chats の出力から対象チャットの id を選び、

meeting_chat_id = "19:meeting_YzkyZGE0NmItN2Y4Zi00NmJkLTgxYzktZDlmZGY3OTM2MGRm@thread.v2"

meeting_chat_messages = get_chat_messages(meeting_chat_id)

# のように呼び出します。

GET https://graph.microsoft.com/v1.0/me/onlineMeetings ? $filter= JoinWebUrl eq 'https://teams.microsoft.com/l/meetup-join/19%3ameeting_YzkyZGE0NmItN2Y4Zi00NmJkLTgxYzktZDlmZGY3OTM2MGRm%40thread.v2/0?context=%7b%22Tid%22%3a%2258970528-d083-476f-9082-2d21bd11a800%22%2c%22Oid%22%3a%22a52ac978-0df2-4e13-91e9-a1571eecde73%22%7d'
  -> 200
{"@odata.context":"https://graph.microsoft.com/v1.0/$metadata#users('a52ac978-0df2-4e13-91e9-a1571eecde73')/onlineMeetings","value":[{"id":"MSphNTJhYzk3OC0wZGYyLTRlMTMtOTFlOS1hMTU3MWVlY2RlNzMqMCoqMTk6bWVldGluZ19Zemt5WkdFME5tSXROMlk0WmkwME5tSmtMVGd4WXprdFpEbG1aR1kzT1RNMk1HUm1AdGhyZWFkLnYy","creationDateTime":"2026-01-05T02:42:32.1733565Z","startDateTime":"2026-01-05T03:00:00Z","endDateTime":"2026-01-05T03:30:00Z","joinUrl":"https://teams.microsoft.com/l/meetup-join/19%3ameeting_YzkyZGE0NmItN2Y4Zi00NmJkLTgxYzktZDlmZGY3OTM2MGRm%40thread.v2/0?context=%7b%22Tid%22%3a%2258970528-d083-476f-9082-2d21bd11a800%22%2c%22Oid%22%3a%22a52ac978-0df2-4e13-91e9-a1571eecde73%